# Projeto 1 - Ciência dos Dados

Nome: Eduardo Selber Castanho

Nome: Henrique Fazzio Badin

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [181]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [182]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\henri\OneDrive\Desktop\PYTHON\CDados\Trabalho 1\22-2a-cd-p1-grupo_henriquefb3\notebooks


Carregando a base de dados com as notícias classificadas manualmente:

In [183]:
filename = 'dados.xlsx'

In [184]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Titulo,Descrição,Pagina,Target,Unnamed: 4,Unnamed: 5
0,CoinFLEX demite funcionários em meio a disputa...,A conturbada corretora de criptomoedas CoinFLE...,12,1,NaN,283.0
1,O que dar de presente neste Natal para alguém ...,Está em busca de um presente legal para um ent...,83,0,NaN,NaN
2,Direto de El Salvador: duas perguntas para o m...,O maximalista de bitcoin Max Keiser abriu as a...,91,0,NaN,NaN
3,Lamborghini de R$ 630 mil do falso Rei do Bitc...,Começa na próxima sexta-feira (22) o leilão de...,15,0,NaN,NaN
4,Nubank usou R$ 140 milhões para comprar até 10...,"Com R$ 14 bilhões em caixa, conforme o último ...",35,1,NaN,NaN


In [185]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Titulo,Descrição,Pagina,Target
0,O estranho caso dos blocos ‘vazios’ na blockch...,Um evento incomum aconteceu na rede do Bitcoin...,57,0
1,Alívio do bitcoin sugere que pior fase do inve...,"Após um mês de consolidação, o preço do bitcoi...",13,0
2,Lei que proíbe pagamentos com bitcoin na Rússi...,"O Parlamento da Rússia, conhecido como Duma, a...",27,0
3,Manhã Cripto: Projeto de Lei que regula cripto...,Com a regulação dos criptoativos no foco de in...,30,1
4,A semana das criptomoedas: ethereum (ETH) em d...,Embora muitas criptomoedas tenham caído nos úl...,16,1


___
## Classificador automático


As criptomoedas é um assunto que anda muito em pauta atualmente, gerando cada vez mais interesse das pessoas em investi-las, mas como todo investidor precisa ser consciente e para isso é necessário acompanhar notícias sobre as atualizações do valor dessas moedas, porém existe muitas notícias, que embora sejam desse assunto não mencionam diretamente o valor delas. Desse modo, decidimos criar um classificador de notícias para que investidores possam analisar quais notícias são relevantes na hora de tomar uma decisão se vão investir nelas ou não.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

1) Primeiramente, integrasse todos os títulos de notícias presentes na planilha de treinamento em um texto utilizando-se um ***for***.

In [186]:
texto=''
for i in range(0,540):
    texto+=train['Titulo'][i]+' '

2) Após isso, cria-se uma função para fazer a limpeza de um texto.

In [187]:
import re 


def cleanup(text):

    punctuation = '[!-.:?;]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

3) Em seguida, aplica-se a função ***cleanup()***, criada no passo 1, no texto desenvolvido no passo 2. Faz-se isso para retirar todas as "impurezas" do texto que podem afetar o aprendizado do classificador.

In [188]:
texto_f=cleanup(texto)

4) Em sequência, fez-se um texto contendo todos os títulos de notícias consideradas relevantes e, logo após isso, um texto que continha todos os textos marcados como irrelevantes.

In [189]:
dados_relevantes=train.loc[train['Target']==1,:]
texto_relevantes=''
for i in dados_relevantes['Titulo']:
    texto_relevantes+=i+' '
texto_relevantes_f=cleanup(texto_relevantes)

dados_irrelevantes=train.loc[train['Target']==0,:]
texto_irrelevantes=''
for i in dados_irrelevantes['Titulo']:
    texto_irrelevantes+=i+' '
texto_irrelevantes_f=cleanup(texto_irrelevantes)

5) Após isso, criaram-se duas tabelas, uma com todas as palavras relevantes e uma com todas as irrelevantes. Dessa forma, a partir da função ***value.counts()***, é possível saber a quantidade de vezes que uma palavra aparece no texto, tanto em valores brutos como em valores relativos.

In [190]:
#Tabela com todas as palavras presente no texto de palavras relevantes:

todas_dados_relevantes=texto_relevantes_f.split()
serie_relevante = pd.Series(todas_dados_relevantes)

#Tabela que apresenta o valor total de vezes que uma palavra aparece na tabela construída acima:

tabela_relevante=serie_relevante.value_counts()

#Tabela que apresenta o valor relativo de vezes que uma palavra aparece no texto de palavras relevantes:

tabela_relevante_rel=serie_relevante.value_counts(True)


In [191]:
#Tabela com todas as palavras presente no texto de palavras irrelevantes:

todas_dados_irrelevantes=texto_irrelevantes_f.split()
serie_irrelevante = pd.Series(todas_dados_irrelevantes)

#Tabela que apresenta o valor total de vezes que uma palavra aparece na tabela construída acima:

tabela_irrelevante=serie_irrelevante.value_counts()

#Tabela que apresenta o valor relativo de vezes que uma palavra aparece no texto de palavras irrelevantes:

tabela_irrelevante_rel=serie_irrelevante.value_counts(True)


6) Faz-se a mesma coisa para o texto que possui todas as palavras.

In [192]:
todas_texto=texto_f.split()
serie_todas=pd.Series(todas_texto)
tabela_todas=serie_todas.value_counts()
tabela_todas_rel=serie_todas.value_counts(True)

7) No suavizador de Laplace além de ter que dividir a quantidade da palavra aparecer mais um pelo total de palavras da categoria, como relevante, por exemplo, também é necessário dividir pelo total de termos dos títulos em que os elementos das frases aparecem. Desse modo, foi criada função ***matrizes()***, a qual irá incluir várias listas, que cada uma apresentará uma lista de quais palavra estão inclusas no título, facilitando na hora de realizar o processo de análise de quais títulos apresentam algum termo presente na frase que será classificada. Essa função foi feita com a utilização de dois ***while***, seno o primeiro para adicionar os títulos da base de dados em uma lista vazia e o segundo para fazer o ***cleanup()*** da lista e o split dela para assim adicioná-la na matriz.

In [193]:
def matrizes(dados):
    i=0
    lista=[]
    while i<len(dados):
        lista.append(dados[i])
        i+=1
    matriz=[] 
    for i in lista:
        frase_ma=cleanup(i)
        frase_matriz=frase_ma.split()
        matriz.append(frase_matriz)
    return matriz
matriz=matrizes(train['Titulo'])

8) Por fim, desenvolve-se o código de uma função relativo ao cálculo da probabilidade de um texto ser considerado relevante ou irrelevante adicionando um mecanísmo chamado ***Suavizador de Laplace***. Basicamente, esse código permitirá o classificador criar uma previsão em relação à qual a probabilidade de um texto ser considerado relevante ou irrelevante mesmo que ele apresente palavras que não se encontram na base de dados da planilha ***Treinamento***. Caso esse suavizador não seja adicionado ao código e uma palavra que não está na base de dados seja escolhida, a probabilidade de esse texto ser tanto relevante como irrelevante é nula, o que resultaria num erro.

In [194]:
def classificador(frase):
    frase_2=cleanup(frase)
    frase_f=frase_2.split()
    probFraseDadoR=1
    x=0
    y=0
    probFraseDadoI=1
    lista=[]
    tanafrase=True
    tanafrasei=True
    for i in frase_f:
        if i not in tabela_relevante:
            tanafrase=False
            
        if i not in tabela_irrelevante:
            tanafrasei=False
        
    for i in frase_f:
        
        for c in matriz:
            if i in c:
                
                lista.append(c)
               
    res = []
    track = []
    count = 0
  
    for sub in lista:
        res.append([]);
        for ele in sub:
            if ele not in track:
                res[count].append(ele)
                track.append(ele)
    c=0
    for number in res:
        c+=len(number)

    for word in frase_f:
        if not tanafrase:
            if word in tabela_relevante_rel:
                x=(tabela_relevante_rel[word]*len(todas_dados_relevantes))+1
                probFraseDadoR=probFraseDadoR*(x/(len(todas_texto)+c))
            else:
                probFraseDadoR=probFraseDadoR*(1/(len(todas_texto)+c))
        if not tanafrasei:
            if word in tabela_irrelevante_rel:
                y=(tabela_irrelevante_rel[word]*len(todas_dados_irrelevantes))+1
                probFraseDadoI=probFraseDadoI*(y/(len(todas_texto)+c))
            else:
                probFraseDadoI=probFraseDadoI*(1/(len(todas_texto)+c))
            
        if tanafrasei:
            probFraseDadoI=probFraseDadoI*tabela_irrelevante_rel[word]
    
        if tanafrase:
            probFraseDadoR=probFraseDadoR*tabela_relevante_rel[word]

    if probFraseDadoR>probFraseDadoI:
        return ('Relevante')
    else:
        return ('Irrelevante')

#### Extra

Teste para observar se o funcionamento está correto com uma notícia aleatória da internet:

In [195]:
classificador('Goleiro Bruno diz que perdeu R$ 50 mil para Faraó do Bitcoin')

'Irrelevante'

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [196]:
matriz2=matrizes(test['Titulo'])
listax=[]

test['Target teste']=test['Target']

listax=[]
c=0
test['Target teste']=test['Target']

resultado=''
for i in matriz2:
    frase_teste=''
    for letra in i:
        frase_teste+=letra+' '
    resultado=classificador(frase_teste)
    
    if resultado=='relevante':
        listax.append(1)
        test['Target teste'][c]=1
        
    elif resultado=='Irrelevante':
        test['Target teste'][c]=0
        listax.append(0)
    c+=1


C:\Users\henri\AppData\Local\Temp\ipykernel_27332\1794038698.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Target teste'][c]=0


O teste de qualidade, observa quantas vezes a categoria da target feita manualmente coincidiu com a target feita pelo código. Esse processo foi feito a partir da criação de um contador e da utilização de um ‘while’ que iria percorrer toda a base de dados e quando o valor das target coincidiam, que para ver isso foi utilizado apenas um ‘if’, somava 1 no contador. No final dividia o valor do contador pelo total de linhas da base de dados, de tal modo que obtida a porcentagem de confiança, ou seja, de quantas vezes o classificador acertou. 

In [197]:
count=0
certas=0
while count<len(test):
    if test['Target'][count]==test['Target teste'][count]:
        certas+=1
    count+=1
print('A porcentagem de acerto da previsão do código é de: {:.2f}%'.format((certas/count)*100))

A porcentagem de acerto da previsão do código é de: 95.56%


___
### Concluindo

Concluindo, o classificador de Naive-Bayes, apesar de apresentar uma boa precisão quando aprimorado com o processo de suavização de Laplace, porém, ele ainda assim apresenta algumas limitações.

A primeira limitação que esse sistema demonstra é o de não ser capaz de gerar mais amostras de treinamento por sí só, já que, como sua base de dados é limitada, os títulos girariam sempre entorno de uma mesma base, o que não permite uma grande expansão na sua capacidade de classificação.

Além disso, o classificador pode ser melhorado para ser mais preciso em suas previsões, como, por exemplo, com a implementação de um código de stemização das palavras ou com a utilização de um código de lematização. 

A Stemização é um processo no qual as palavras estudadas são reduzidas ao seu radical. Esse processo pode ser muito útil para o classificador, uma vez que, ao invés de estar comparando palavras completas que podem possuir diferenças em apenas uma letra mas apresentam a mesma informação, o código faria uma comparação entre o radical dessas palavras, permitindo uma precisão maior. Por exemplo, se o título de uma notícia relevante fosse "Cachorros para adoção nas faculdades" e o título entregado ao classificador fosse "Cachorro pra adoção na faculdade", o classificador normalmente daria a notícia como irrelevante, porém, com a stemização, essa notícia entraria para a lista de relevantes.

Já na Lematização também tenta-se reduzir as palavras à sua raiz, mas esse corte sempre gera uma palavra qu existe na gramática. Ou seja, todas as palavras seriam reduzidas à palavra que origina ela. Por exemplo, a palavra lançamento seria reduzida para lançar, perfurmar para perfume, etc. Dessa forma, assim como na stemização, caso houvesse alguma alteração pequena entre os títulos analisados, não faria qualquer diferença para o cálculo da probabilidade final. 

Para implementar entender como essas modificações, é possível encontrar uma explicação no site da empresa Alura, por exemplo, assim como em diversos outros sites na internet.

Outorssim, é que o classificador pode ser melhorado através de uma expansão da base de dados, ou seja, da tabela de relevância e irrelevância das moedas, de tal modo que seja instalado mais base de dados de outras fontes de notícia, como o ‘isto é’. Além disso, também pode ser melhorado ao expandir os conceitos das targets em muito relevante ou muito irrelevante, para dar uma possibilidade do usuário em saber quais ele deve dar ainda mais atenção e quais podem ser completamente ignoradas.

Por fim, podemos encontrar também muitos pontos positivos no classificador de Naive-Bayes. Um deles, por exemplo, é na sua facilidade de implementação em diversos contextos. Entre esses cenários estão, por exemplo: Possível auxílio médico de classificações de doenças para o paciente de acordo com as informações de sintomas que ele apresenta; Classificação de possíveis fake news por meio da relação entre palavras chaves presentes na notícia analisada e outras notícias falsas; Pode até mesmo ser util para auxiliar na previsão do tempo, uma vez que pode comparar as condições climáticas do dia analisado, como umidade, pressão e temperatura, e relacionar com o que aconteceu em outros dias que apresentaram circunstâncias similares de uma base de dados. 

___
### Qualidade do Classificador a partir de novas separações das notícias entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas notícias. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas caterogias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por caterogia (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**